In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45730
2,Huelva,Confirmados PDIA 14 días,303
3,Huelva,Tasa PDIA 14 días,"59,044760995381644"
4,Huelva,Confirmados PDIA 7 días,134
5,Huelva,Tasa PDIA 7 dias,"26,112204532611024"
6,Huelva,Total Confirmados,45932
7,Huelva,Curados,44790
8,Huelva,Fallecidos,418


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45730.0


/tmp/ipykernel_3099/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12592.0


/tmp/ipykernel_3099/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3099/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3099/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3099/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 6849 personas en los últimos 7 días 

Un positivo PCR cada 3425 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45730.0,134.0,303.0,513170.0,26.112205,59.044761,26.0
Huelva-Costa,27053.0,68.0,158.0,289548.0,23.484880,54.567809,16.0
Condado-Campiña,14216.0,51.0,111.0,156231.0,32.643970,71.048640,7.0
Huelva (capital),12592.0,21.0,42.0,143837.0,14.599860,29.199719,6.0
Isla Cristina,3140.0,9.0,13.0,21393.0,42.069836,60.767541,4.0
Cartaya,2041.0,25.0,61.0,20083.0,124.483394,303.739481,3.0
Moguer,1987.0,12.0,25.0,21867.0,54.877212,114.327525,3.0
Sierra de Huelva-Andévalo Central,4035.0,14.0,33.0,67391.0,20.774287,48.967963,3.0
Lepe,3027.0,4.0,11.0,27880.0,14.347202,39.454806,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villarrasa,229.0,3.0,7.0,2211.0,135.685210,316.598824,0.0
Cartaya,2041.0,25.0,61.0,20083.0,124.483394,303.739481,3.0
Manzanilla,144.0,3.0,6.0,2118.0,141.643059,283.286119,0.0
Valverde del Camino,858.0,8.0,20.0,12750.0,62.745098,156.862745,0.0
Villalba del Alcor,502.0,2.0,5.0,3366.0,59.417706,148.544266,0.0
Palos de la Frontera,1047.0,10.0,17.0,11742.0,85.164367,144.779424,1.0
Moguer,1987.0,12.0,25.0,21867.0,54.877212,114.327525,3.0
Bonares,464.0,1.0,6.0,6060.0,16.501650,99.009901,0.0
Paterna del Campo,211.0,1.0,3.0,3457.0,28.926815,86.780445,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bonares,464.0,1.0,6.0,6060.0,16.501650,99.009901,0.0,0.166667
Ayamonte,2090.0,3.0,14.0,21104.0,14.215315,66.338135,0.0,0.214286
Punta Umbría,1274.0,3.0,11.0,15355.0,19.537610,71.637903,1.0,0.272727
Paterna del Campo,211.0,1.0,3.0,3457.0,28.926815,86.780445,0.0,0.333333
Lepe,3027.0,4.0,11.0,27880.0,14.347202,39.454806,2.0,0.363636
Almonte,2307.0,7.0,19.0,24507.0,28.563268,77.528869,0.0,0.368421
Villalba del Alcor,502.0,2.0,5.0,3366.0,59.417706,148.544266,0.0,0.400000
Valverde del Camino,858.0,8.0,20.0,12750.0,62.745098,156.862745,0.0,0.400000
Cartaya,2041.0,25.0,61.0,20083.0,124.483394,303.739481,3.0,0.409836
